# First models

Here, we build a modelling skeleton that we can hang further developments off of. 

In [37]:
import scipy
import pandas as pd
import numpy as np
import os
from os.path import join
from pprint import pprint

import xeek
import xeek.features as features

from sklearn.impute import KNNImputer

%matplotlib inline
from importlib import reload
reload(xeek)
reload(features)

<module 'xeek.features' from '/home/ubuntu/projects/xeek-wel-logs-facies/xeek/features.py'>

## Data import

In [4]:
df_train = pd.read_csv(xeek.raw_train_filepath, sep=";")

In [5]:
df_train.head()

,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,RMED,...,ROP,DTS,DCAL,DRHO,MUDWEIGHT,RMIC,ROPA,RXO,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE
0,15/9-13,494.528,437641.96875,6470972.5,-469.501831,NORDLAND GP.,NaN,19.480835,NaN,1.611410,...,34.636410,NaN,NaN,-0.574928,NaN,NaN,NaN,NaN,65000,1.0
1,15/9-13,494.680,437641.96875,6470972.5,-469.653809,NORDLAND GP.,NaN,19.468800,NaN,1.618070,...,34.636410,NaN,NaN,-0.570188,NaN,NaN,NaN,NaN,65000,1.0
2,15/9-13,494.832,437641.96875,6470972.5,-469.805786,NORDLAND GP.,NaN,19.468800,NaN,1.626459,...,34.779556,NaN,NaN,-0.574245,NaN,NaN,NaN,NaN,65000,1.0
3,15/9-13,494.984,437641.96875,6470972.5,-469.957794,NORDLAND GP.,NaN,19.459282,NaN,1.621594,...,39.965164,NaN,NaN,-0.586315,NaN,NaN,NaN,NaN,65000,1.0
4,15/9-13,495.136,437641.96875,6470972.5,-470.109772,NORDLAND GP.,NaN,19.453100,NaN,1.602679,...,57.483765,NaN,NaN,-0.597914,NaN,NaN,NaN,NaN,65000,1.0


In [6]:
df_test = pd.read_csv(xeek.raw_test_filepath, sep=";")

In [7]:
df_test.head()

,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,RMED,...,SP,BS,ROP,DTS,DCAL,DRHO,MUDWEIGHT,RMIC,ROPA,RXO
0,15/9-14,480.628001,423244.50000,6461862.5,-455.624420,NORDLAND GP.,NaN,19.2031,NaN,1.613886,...,35.525719,NaN,96.461990,NaN,NaN,-0.538873,0.130611,NaN,NaN,NaN
1,15/9-14,480.780001,423244.50000,6461862.5,-455.776428,NORDLAND GP.,NaN,19.2031,NaN,1.574376,...,36.158520,NaN,96.454399,NaN,NaN,-0.539232,0.130611,NaN,NaN,NaN
2,15/9-14,480.932001,423244.50000,6461862.5,-455.928436,NORDLAND GP.,NaN,19.2031,NaN,1.436627,...,36.873703,NaN,96.446686,NaN,NaN,-0.540830,0.130611,NaN,NaN,NaN
3,15/9-14,481.084001,423244.50000,6461862.5,-456.080444,NORDLAND GP.,NaN,19.2031,NaN,1.276094,...,37.304054,NaN,161.170166,NaN,NaN,-0.543943,0.130611,NaN,NaN,NaN
4,15/9-14,481.236001,423244.53125,6461862.5,-456.232422,NORDLAND GP.,NaN,19.2031,NaN,1.204704,...,37.864922,NaN,172.489120,NaN,NaN,-0.542104,0.130611,NaN,NaN,NaN


## Modelling approach

For this first approach, we generate an "end-to-end" model with a deliberately simple approach. Our goal here is not to produce an impressive model, but moreso to generate a skeleton to hang further work off of.

Our first modelling approach will:

* Use only features that appear in most of the wells
* Cross-validate on a well basis
* In-fill missing values with a mean-per-well value

### Feature selection

We retrieve only the features that are present in all the majority of the well logs:

In [8]:
universal_features = features.features_mostly_present(df_train, presence_threshold=.8)
pprint(universal_features)

['WELL',
 'DEPTH_MD',
 'X_LOC',
 'Y_LOC',
 'Z_LOC',
 'GROUP',
 'FORMATION',
 'CALI',
 'RMED',
 'RDEP',
 'RHOB',
 'GR',
 'DTC',
 'DRHO',
 'FORCE_2020_LITHOFACIES_LITHOLOGY',
 'FORCE_2020_LITHOFACIES_CONFIDENCE']


How does this compare to our test dataset?

In [9]:
universal_features_test = features.features_mostly_present(df_test, presence_threshold=.8)
pprint(universal_features_test)

['WELL',
 'DEPTH_MD',
 'X_LOC',
 'Y_LOC',
 'Z_LOC',
 'GROUP',
 'FORMATION',
 'CALI',
 'RMED',
 'RDEP',
 'RHOB',
 'GR',
 'NPHI',
 'PEF',
 'DTC',
 'DRHO']


Our test dataset is a superset, so we are good to work with this limited dataset.

We will further limit our features to continuous features.

In [10]:
feature_columns = [feature for feature in universal_features if
                   (feature in features.well_log_features)]
columns = feature_columns + features.target + ["WELL"]

In [11]:
columns

['CALI',
 'RMED',
 'RDEP',
 'RHOB',
 'GR',
 'DTC',
 'FORCE_2020_LITHOFACIES_LITHOLOGY',
 'WELL']

In [12]:
df_train_limited = df_train.loc[:, columns]

In [13]:
df_train_limited

,CALI,RMED,RDEP,RHOB,GR,DTC,FORCE_2020_LITHOFACIES_LITHOLOGY,WELL
0,19.480835,1.611410,1.798681,1.884186,80.200851,161.131180,65000,15/9-13
1,19.468800,1.618070,1.795641,1.889794,79.262886,160.603470,65000,15/9-13
2,19.468800,1.626459,1.800733,1.896523,74.821999,160.173615,65000,15/9-13
3,19.459282,1.621594,1.801517,1.891913,72.878922,160.149429,65000,15/9-13
4,19.453100,1.602679,1.795299,1.880034,71.729141,160.128342,65000,15/9-13
...,...,...,...,...,...,...,...,...
1170506,8.423170,NaN,NaN,2.527984,77.654900,NaN,30000,7/1-2 S
1170507,8.379244,NaN,NaN,2.537613,75.363937,NaN,65030,7/1-2 S
1170508,8.350248,NaN,NaN,2.491860,66.452843,NaN,65030,7/1-2 S
1170509,8.313779,NaN,NaN,2.447539,55.784817,NaN,65030,7/1-2 S


### Missingness

Based on this process, we will briefly review the data missingness:

In [14]:
(df_train
.groupby("WELL")
.aggregate(lambda s: s.isna().sum() / len(s))
.sort_values("CALI", ascending=False))

,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,RMED,RDEP,...,ROP,DTS,DCAL,DRHO,MUDWEIGHT,RMIC,ROPA,RXO,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE
WELL,,,,,,,,,,,,,,,,,,,,,
35/8-4,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.161683,0.000000,0.000000,...,1.0,1.000000,1.000000,0.000000,1.000000,0.162052,0.000000,1.000000,0,0.001107
36/7-3,0.0,0.000827,0.000827,0.000827,0.000000,0.000000,0.982359,1.000000,0.001103,0.000827,...,1.0,1.000000,1.000000,0.042448,1.000000,1.000000,0.000551,1.000000,0,0.000276
35/11-12,0.0,0.000000,0.000000,0.000000,0.000000,0.124529,0.804521,1.000000,0.003924,0.000000,...,0.0,0.817235,1.000000,0.822677,1.000000,1.000000,1.000000,1.000000,0,0.000157
31/5-4 S,0.0,0.158299,0.158299,0.158299,0.101821,0.604826,0.796270,0.797773,0.796270,0.158299,...,1.0,1.000000,1.000000,0.786371,1.000000,1.000000,1.000000,0.796270,0,0.000088
33/9-1,0.0,0.000000,0.000000,0.000000,0.000000,0.099491,0.689027,0.882799,0.000000,0.000000,...,1.0,1.000000,1.000000,0.398541,1.000000,1.000000,1.000000,1.000000,0,0.000058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25/8-5 S,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000617,0.005481,0.000000,...,0.0,0.856742,0.007331,0.034599,0.010071,0.497122,1.000000,0.000000,0,0.000000
25/7-2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010176,0.002638,0.000000,...,0.0,1.000000,0.000000,0.004523,1.000000,0.000000,1.000000,0.006324,0,0.000042
25/6-3,0.0,0.000000,0.000000,0.000000,0.000000,0.014101,0.000000,0.316229,0.003421,0.000000,...,0.0,0.000000,1.000000,1.000000,0.327993,0.316229,1.000000,1.000000,0,0.000167


Most of our wells have relatively complete data, but there are some outliers. `16/11-1 ST3` is particularly messy. In fact, were our threshold for whether a feature is "present" slightly lower, we probably wouldn't include `X_LOC`, `Y_LOC`, `Z_LOC`, and `RDEP` as features.

## Pipeline

We wish to design a pipeline for our model. We will:

* Split our data into X and Y inputs
* Impute the missing values
* Build a basic linear classification model
* Judge this by our custom metric

In [15]:
from sklearn import compose, pipeline, preprocessing, linear_model, impute, model_selection

### Column split out

In [16]:
def split_into_x_y_groups(df, x_columns, y_column="FORCE_2020_LITHOFACIES_LITHOLOGY", group_column=["WELL"]):
    X = df[x_columns].to_numpy()
    if y_column is None:
        Y = None
    else:
        Y = df[y_column].map(features.lithology_mapping).to_numpy()
    group = df[group_column].to_numpy()
    
    return X, Y, group

In [17]:
X, Y, group = split_into_x_y_groups(df_train, feature_columns, features.target[0], group_column=["WELL"])

### Build a pipeline

In [18]:
classification_pipeline = pipeline.make_pipeline(
    preprocessing.StandardScaler(),
    impute.SimpleImputer(strategy="median"),
    linear_model.SGDClassifier()
)

In [19]:
kfold = model_selection.GroupKFold(n_splits=5)

In [20]:
A = np.load(join(xeek.external_data_dir, 'penalty_matrix.npy'))

In [21]:
def score(y_true, y_pred):
    S = 0.0
    y_true = y_true.astype(int)
    y_pred = y_pred.astype(int)
    for i in range(0, y_true.shape[0]):
        S -= A[y_true[i], y_pred[i]]
    return S/y_true.shape[0]

In [22]:
def scorer(estimator, X, y):
    """
    Utility function for cross validation.
    """
    y_hat = estimator.predict(X)
    return score(y, y_hat)

In [23]:
scores = model_selection.cross_val_score(
    estimator=classification_pipeline,
    X = X,
    y = Y,
    groups=group,
    cv=kfold,
    n_jobs=-1,
    scoring=scorer)

In [24]:
scores

array([-1.02701738, -1.02569781, -0.93158557, -1.09664402, -1.00195529])

## Full training job

In [25]:
classification_pipeline.fit(X, Y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('simpleimputer', SimpleImputer(strategy='median')),
                ('sgdclassifier', SGDClassifier())])

## Output results to standard format

In [26]:
X_test, Y_test, group_test = split_into_x_y_groups(df_test, feature_columns, y_column=None, group_column="WELL")

In [27]:
Y_test_hat = classification_pipeline.predict(X_test)

In [47]:
Y_test_hat = np.vectorize(xeek.features.lithology_inverse_mapping.get)(Y_test_hat)
np.savetxt(join(xeek.processed_data_dir, 'test_predictions.csv'), Y_test_hat, header='lithology', comments='', fmt='%i')